#### This notebook is for IBM certificate capstone project on Coursera.

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs

#### Here I am using Requests library to retrieve webpage from wikipedia

In [2]:
resp = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
pageText = resp.text
soup = bs(pageText, 'html.parser')

In [3]:
table = soup.find_all("table")

In [4]:
tablist = []
for mytable in table:
    table_body = mytable.find('tbody')
    try:
        rows = table_body.find_all('tr')
        for tr in rows:
            cols = tr.find_all('td')
            temp = ""
            for td in cols:
                temp += td.text+","
            tablist.append(temp)
    except:
        print("no tbody")

no tbody


There is some cluttering in start and after of the table while scraping so removing them by using indexing...

In [5]:
tablist = tablist[1:-7]

In [6]:
## Here each row is being put into separate list. This will make is easy to import to dataframe
## in separate columns.

new_tablist = []
for each in tablist:
    each = each.replace('\n','')
    new_tablist.append(each.split(','))


In [7]:
db = pd.DataFrame(new_tablist)

In [8]:
## Giving columns names, the last one is extraneous columns, so will drop it
db.columns = ['PostalCode', 'Borough', 'Neighbourhood','todrop']

In [9]:
db.drop('todrop',axis=1,inplace=True)

In [10]:
## Ignoring those postal areas with Borough = Not assigned
db = db[db['Borough']!='Not assigned']

In [11]:
db.head()

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [12]:
## see the index is disturbed after, so resetting it and dropping the previous index columns
db.reset_index(inplace=True)
db.drop('index',axis=1,inplace=True)

In [13]:
## we need onyl unique PostalCode and want to merge the duplicate Neighbourhood.
db2 = db.groupby(['PostalCode']).agg({'Borough':'first', 'Neighbourhood': ', '.join}).reset_index()

In [14]:
db2.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [15]:
## Checking if our logic was right and the answer shows that two neighbour are merged correctly
db2[db2['PostalCode']=='M5A']

,PostalCode,Borough,Neighbourhood
53,M5A,Downtown Toronto,"Harbourfront, Regent Park"


In [16]:
## Checking if there are such PostalCodes with Neighbourhood set to Not assigned
db2[db2['Neighbourhood']=='Not assigned']

,PostalCode,Borough,Neighbourhood
85,M7A,Queen's Park,Not assigned


In [17]:
## np.where is a good option to replace, see the documentation
db2['Neighbourhood'] = np.where(db2['Neighbourhood']!='Not assigned', db2['Neighbourhood'], db2['Borough'])

In [18]:
## checking if our logic was right, turns out that Neighbourhood is correctly replaced by Borough value
db2[db2['PostalCode']=='M7A']

,PostalCode,Borough,Neighbourhood
85,M7A,Queen's Park,Queen's Park


In [19]:
db2.shape

(103, 3)

In [32]:
## Just creating dummy columns, they will get replaced later
db2['Latitude'] = db2['PostalCode']
db2['Longitude'] = db2['PostalCode']

In [20]:
## reading csv file for geospatial data
geos_c = pd.read_csv('Geospatial_Coordinates.csv')

In [31]:
geos_c

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [33]:
db2['Latitude'] = np.where(db2['PostalCode']!=geos_c['Postal Code'], db2['Latitude'], geos_c['Latitude'])
db2['Longitude'] = np.where(db2['PostalCode']!=geos_c['Postal Code'], db2['Longitude'], geos_c['Longitude'])

In [34]:
db2

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.8067,-79.1944
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.7845,-79.1605
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.7636,-79.1887
3,M1G,Scarborough,Woburn,43.771,-79.2169
4,M1H,Scarborough,Cedarbrae,43.7731,-79.2395
5,M1J,Scarborough,Scarborough Village,43.7447,-79.2395
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.7279,-79.262
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.7111,-79.2846
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.7163,-79.2395
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.6927,-79.2648
